## General imports and definitions

In [ ]:
from framed.io_utils.sbml import load_cbmodel
from framed.analysis.simulation import pFBA, FBA
from framed.analysis.variability import FVA
from framed.core.transformation import simplify
import pandas as pd
import seaborn as sns
%pylab inline
sns.set_style("whitegrid")

In [ ]:
%run '../src/gpr_mapping.py'

## Load and transform iAF1260

In [ ]:
gs_model = load_cbmodel('../../models/iAF1260_reduced.xml', flavor='cobra')
gs_model_ext = transform(gs_model, inplace=False)

## Run FVA for iAF1260

In [ ]:
%%time
min_growth = 0.1
var = FVA(gs_model, obj_percentage=min_growth)
loops = [r_id for r_id, (lb, ub) in var.items() if lb is None or ub is None]

In [ ]:
%%time
var_loops = FVA(gs_model, obj_percentage=min_growth, reactions=loops,
                loopless=True, internal=loops)
var.update(var_loops)

In [ ]:
%%time
constraints = gs_model_ext.convert_constraints(dict(var))
synth_reactions = [r_id for r_id in gs_model_ext.reactions if r_id.startswith('u_')]
var_synth = FVA(gs_model_ext, obj_percentage=min_growth, 
                reactions=synth_reactions, constraints=constraints)

## Plot Results

In [ ]:
figure(figsize=(7,4))
x = range(len(synth_reactions))
lb, ub = array(var_synth.values()).T
ub1, ub2 = ub[lb > 0], ub[lb == 0]
lb1, lb2 = lb[lb > 0], lb[lb == 0]
ind1, ind2 = argsort(-ub1), argsort(-ub2)
lb = concatenate((lb1[ind1], lb2[ind2]))
ub = concatenate((ub1[ind1], ub2[ind2]))
bar(x, ub, bottom=lb, width=1, linewidth=0)
xlim((0, len(x)))
xticks([])
yscale('symlog', linthreshy=1e-5)
yticks([0, 1e-4, 1e-2, 1e0, 1e2, 1e4])
ylabel('enzyme usage variability (mmol/gDW/h)')
text(15, -6e-6, 'essential genes')
text(450, -6e-6, 'non-essential genes')
savefig('../results/essentiality/essentiality.png', dpi=150)